In [419]:
import pandas as pd
import glob, os
import numpy as np

# Divisione train/test set e calcolo media su test

Voglio prendere il mio train set che è dato dal match con nuovi campi aggiuntivi. 
Questi campi sono la differenza tra le statistiche in quel match della squadra di casa e quelle della squadra in trasferta.

Nel test set invece prendo i csv relativi alle due squadre che si affrontano, prendo la giornata e calcolo la media dei match precedenti. Successivamente faccio la sottrazione.

In [420]:
rename_fields = {
    'Squadra': 'team',
    'Rf': 'goals',
    'Tiri': 'total_shots',
    'Tiri.1': 'shots_on_target',
    'Rigori': 'goals_on_penalty',
    'Rig T': 'total_penalties',
    'Compl': 'completed_passings',
    'Tent': 'total_passings',
    'Angoli': 'corners',
    'Poss.': 'percentage_possession',
    'Falli': 'fouls',
    'Amm.': 'yellow_cards',
    'Esp.': 'red_cards',
    'Data': 'date',
    'Girone': 'matchday',
}

In [421]:
dataset = pd.read_csv("SerieA/Season21_22/matches.csv", index_col=0)
matches = dataset[['h_team', 'a_team', 'date', 'matchday', 'result']]
matches.to_csv('only_match.csv')

In [422]:
#percentage = len(matches)
percentage = len(matches)*80/100 
train = matches[:int(percentage)]
test = matches[int(percentage):]


In [423]:
#train = matches[matches['date']<'2022-03-18']
#test = matches[(matches['date']>='2022-03-18')]# & (matches['date']<'2022-05-23')]


In [424]:
columns = ['goals', 'total_shots', 'shots_on_target', 'goals_on_penalty', 'total_penalties', 'completed_passings', 'total_passings', 'corners','percentage_possession', 'fouls', 'yellow_cards', 'red_cards']

In [425]:
stats_of_all_teams = {}
for file in sorted(os.listdir("SerieA/Season21_22/Matches/")):
    if file.endswith(".csv"):
        team_name = file.replace('.csv', '')
        team_stats = pd.read_csv("SerieA/Season21_22/Matches/"+file, index_col=0)
        team_stats = team_stats.rename(columns=rename_fields) #rinomino le colonne per avere una nomenclatura 
        
        team_stats = team_stats.drop(columns=['Stadio', 'Avversario', 'Rs'])
        dataset = team_stats.drop(columns=['team', 'matchday', 'Risultato'])
        dataset['date'] = pd.to_datetime(dataset['date'], format='%d-%m-%Y')

        stats_of_all_teams[team_name] = dataset

Vado a popolare il train set con le statistiche corrette. Prendo le statistiche per la squadra di casa, per la squadra di trasferta e calcolo la differenza

In [426]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

Popolo il train set con le statistiche

In [427]:
for i, match_value in train.iterrows():
    home, away, date = match_value.h_team, match_value.a_team, match_value.date
    #ottengo nel dizionario i due dataset relativi alle squadre che si scontrano
    home_dataset = stats_of_all_teams[home]
    away_dataset = stats_of_all_teams[away]
    #cerco nei dataset le statistiche relative alla partita
    home_row = home_dataset[home_dataset['date'] == date]
    away_row = away_dataset[away_dataset['date'] == date]

    home_row = home_row.drop(columns=['date'])
    away_row = away_row.drop(columns=['date'])
    
    for col in columns: 
        diff = home_row.iloc[0][col] - away_row.iloc[0][col]
        train.at[i, col] = diff


Popolo il test set con le statistiche

In [428]:
for i, match_value in test.iterrows():
    home, away, date = match_value.h_team, match_value.a_team, match_value.date
    #ottengo nel dizionario i due dataset relativi alle squadre che si scontrano
    home_dataset = stats_of_all_teams[home]
    away_dataset = stats_of_all_teams[away]
    #cerco nei dataset le statistiche relative alle partite
    home_row = home_dataset[home_dataset['date'] < date]
    away_row = away_dataset[away_dataset['date'] < date]

    #home_row = home_row.drop(columns=['date'])
    #away_row = away_row.drop(columns=['date'])

    home_values = home_row[columns].mean()
    away_values = away_row[columns].mean()
    
    for col in columns: 
        diff = home_values[col] - away_values[col]
        test.at[i, col] = diff
    

# Calcolo codici

Concateno i due dataframe insieme per poter calcolare i dummy

In [429]:
dataset = train.append(test)

dataset = dataset.reset_index(drop=True) #devo resettare l'index perché altrimenti quando faccio il dummy, viene fatto anche sull'index, in quando train e test set hanno indici comuni

In [430]:
stacked = dataset[['h_team', 'a_team']].stack()
index = stacked.index.get_level_values(0)
result = pd.crosstab(index=index, columns=stacked)
result.index.name = None
result.columns.name = None

dummy_match = pd.get_dummies(dataset
        ,columns = ['h_team']
        ,prefix = ['h']
        )

dummy_match = dummy_match.drop(['a_team'], axis=1)
pos = 0
for col in result.columns:
    dummy_match.insert(pos, col, result[col])
    pos += 1

dataset = dummy_match

In [431]:
concat_to_train = dataset[:len(train)]
concat_to_test = dataset[len(train):]

concat_to_test = concat_to_test.reset_index(drop=True) #dato che avevo settato un indice progressivo nel dataset completo, devo resettar egli indici per il dummy fatto sul train set

In [432]:
for col in concat_to_train.columns:
    train[col] = concat_to_train[col]
    
for col in concat_to_test.columns:
    test[col] = concat_to_test[col]


In [ ]:
from IPython.display import display, HTML

display(HTML(test.to_html()))

In [434]:
label_train = train.result
label_test = test.result

train = train.drop(columns=['h_team', 'a_team', 'date', 'matchday', 'result'])
test = test.drop(columns=['h_team', 'a_team', 'date', 'matchday', 'result'])

## Machine Learning

In [435]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

import sklearn.metrics as mtr

In [436]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=10000), #max_iter di default vale 100, ho dovuto alzarlo se no non converge
    'Support Vector Machine': SVC(),
    #'Multinomial Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

In [437]:
for model_name, model in models.items():
    models[model_name].fit(train, label_train)

predictions = {}
for model_name, model in models.items():
    predictions[model_name] = model.predict(test)

In [438]:
E = []
for estimator, y_pred in predictions.items():
    report = mtr.classification_report(label_test, y_pred, output_dict=True, zero_division=0)
    E.append({
        'Model': estimator, 'Accuracy': report['accuracy'],
        'Avg Precision (macro)': report['macro avg']['precision'],
        'Avg Recall (macro)': report['macro avg']['recall'],
        'Avg F1-score (macro)': report['macro avg']['f1-score'],
        'Avg Precision (weighted)': report['weighted avg']['precision'],
        'Avg Recall (weighted)': report['weighted avg']['recall'],
        'Avg F1-score (weighted)': report['weighted avg']['f1-score']
    })
E = pd.DataFrame(E).set_index('Model', inplace=False)

In [439]:
E

,Accuracy,Avg Precision (macro),Avg Recall (macro),Avg F1-score (macro),Avg Precision (weighted),Avg Recall (weighted),Avg F1-score (weighted)
Model,,,,,,,
Logistic Regression,0.400000,0.472143,0.424786,0.404762,0.517744,0.400000,0.419850
Support Vector Machine,0.505263,0.372457,0.421652,0.371717,0.437658,0.505263,0.440957
Decision Tree,0.431579,0.530327,0.468091,0.437862,0.581651,0.431579,0.450794
Random Forest,0.431579,0.530327,0.468091,0.437862,0.581651,0.431579,0.450794
K-Nearest Neighbors,0.378947,0.368962,0.359687,0.360450,0.404657,0.378947,0.388221
